# Sistemas de Recomendación - Demographic Filtering
Beatriz Cárdaba y Sara Bengoechea

- Librerías
- Data Set
- Teoría
- Definición algortimo
- Pueba de algoritmo


### Librerías

In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt
import seaborn as sns
from ast import literal_eval
from wordcloud import WordCloud, STOPWORDS
import ast
import plotly
import plotly.offline as py

### Data set Utilizado:
Importamos los data set que se va a utilizar para este sistema de recomendación:
 - Movies: nos dará datos genereles de las películas.
 - credits: nos facilita los siguintes datos:
         - cast
         - crew

In [3]:


movies = pd.read_csv("../data/01_raw/movies_metadata.csv")

/opt/anaconda3/envs/rec_sys_env/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning:

Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.



In [4]:
credits = pd.read_csv("../data/01_raw/credits.csv")
credits['id']= credits['id'].astype(str) # pasamos id a tipo string ya el numero es identificativo cono un DNI

In [5]:
movies = movies.merge(credits,on='id') # unimos las dos tablas importadas para tener la info de credits en la tabla movies

In [6]:
movies.dtypes

adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
cast                      object
crew                      object
dtype: object

## 1. Filtrado demográfico
Ofrecen recomendaciones generalizadas a cada usuario, basadas en la popularidad del artículo y/o el género. El sistema recomienda los mismos artículos a usuarios con características demográficas similares. Dado que cada usuario es diferente, se considera que este enfoque es demasiado simple. La idea básica de este sistema es que los artículos que son más populares y aclamados por la crítica tendrán una mayor probabilidad de ser apreciados por el público medio.



**Algoritmo**

Decide una métrica de puntuación para anotar o calificar la película. Calcula la puntuación de cada película.
Ordena las puntuaciones y recomienda la película mejor valorada a los usuarios.


PASO 1

Para calificar las películas podemos usar "voting_average" como una métrica, pero, promediando las mismas podemos llevar la calificación alta a un nivel bajo como 8.9 y sólo 3 votos no pueden considerar mejor que 7.8 como calificación media para 40 votos. Por lo tanto, utilizando la clasificación ponderada de la IMDB que es :

WR = (v/(v+m) R) + (m/(v+m) C)

donde,

v -> cuenta_votos, el número de votos para la película

m -> los votos mínimos requeridos para ser listados en la tabla (percentil 90) = movies['vote_count'].quantile(0.9)

R -> vote_average, es el promedio de la calificación de la película.

C -> el voto medio en todo el informe = películas['vote_average'].mean

La puntuación media de todas las películas es aproximadamente 6 en una escala de 10 (C). El siguiente paso es determinar un valor apropiado para m, el mínimo de votos requeridos para aparecer en la tabla. Usaremos el 90 por ciento como límite. En otras palabras, para que una película aparezca en las listas, debe tener más votos que al menos el 90% de las películas de la lista.

In [7]:
C = movies['vote_average'].mean()  # la media de los votos de todas las películas
C


5.618232129131436

In [8]:
m = movies['vote_count'].quantile(0.9) # establecemos el mínimo de votos requeridos para que aparezxcaa en la tabla
m # usamos aquellos que estén en el percentil 0.9, excluimos el 0.1 que menos votos tiene

160.0

Ahora, necesitamos calcular nuestra métrica para cada película calificada. Para ello, definiremos una función, weighted_rating() y definiremos una nueva puntuación de la película, de la cual calcularemos el valor aplicando esta función a nuestro DataFrame de películas cualificadas

In [9]:

def imbd_weighted_ratings(movies, m=m, C=C):
    v =movies["vote_count"] # el número de votos de cada película
    R =movies["vote_average"]# la calificación de cada pelicula
    left_term = v /(v + m) #  (v/(v+m) R)
    right_term = m /(v + m) # (m/(v+m) 
    return left_term* R + right_term* C # devueleve el cáculo de la funcón completa: WR = (v/(v+m) R) + (m/(v+m) C)


Seleccionamos aquellas peliculas que tienen más de de los votos necesarios para poder ser recomendades, lo hemos calculado con el percentil 90

In [10]:
qualified_movies  = movies.copy().loc[movies["vote_count"] >= m] # aquellas que tengan mas de los votos establecidos en m


PASO 2

Calculamos el score para cada película.

In [11]:
qualified_movies["imdb_score"] = qualified_movies.apply(imbd_weighted_ratings, axis = 1)

PASO 3

Ordenamos las películas en función de su score (imdb_score) de mayor puntuación a menor puntuación media

In [12]:
qualified_movies.sort_values("imdb_score", ascending=False, inplace= True)

Seleccionamos las columnas que nos interesan para visualizar cuando filtramos

In [13]:
filtered_qualified_movies = qualified_movies[['title', 'popularity', 'vote_count','vote_average', "imdb_score"]]

PASO 4

Definimos una función para que nos devuelva la películas mejor calificadas

In [14]:

def get_recommendations(k):
    return filtered_qualified_movies[:k] # nos devuelve la lista de las películas más recomendas. La lista será de tamaño k

In [15]:
get_recommendations(5)

,title,popularity,vote_count,vote_average,imdb_score
314,The Shawshank Redemption,51.6454,8358.0,8.5,8.445870
837,The Godfather,41.1093,6024.0,8.5,8.425439
10345,Dilwale Dulhania Le Jayenge,34.457,661.0,9.1,8.421458
12525,The Dark Knight,123.167,12269.0,8.3,8.265477
2854,Fight Club,63.8696,9678.0,8.3,8.256385



PROBLEMA DEMOGRAPHIC FILTERING -
Los artículos más populares serían los mismos para cada usuario, todos ven las mismas recomendaciones.

__Bibliografía:__
 - https://www.analyticsvidhya.com/blog/2020/11/create-your-own-movie-movie-recommendation-system/
 - https://github.com/Invince11/IMBD-Movie-Recommendation-System/blob/master/Movie-Recommendation.ipynb
 - https://www.kaggle.com/biphili/movie-recommender-system-for-beginners
 - https://medium.com/nerd-for-tech/simple-movie-recommendation-system-using-python-e5df3ce58664
 - https://medium.com/towards-artificial-intelligence/recommendation-system-in-depth-tutorial-with-python-for-netflix-using-collaborative-filtering-533ff8a0e444
 - https://www.ijert.org/research/recommender-systems-types-of-filtering-techniques-IJERTV3IS110197.pdf
 
